## Imports

In [7]:
import requests
import csv
import io
from bs4 import BeautifulSoup
import os
import glob
import pandas as pd

In [2]:
# Lendo a primeira página

In [3]:
embrapa_url = "http://vitibrasil.cnpuv.embrapa.br/index.php"
try:
    re = requests.get(embrapa_url)
    re.raise_for_status()
except requests.RequestException as err:
    print("erro")
if re.status_code != 200:
    raise "Erro de conexão"
soup = BeautifulSoup(re._content)
# páginas
pages = {}
for i in soup.find_all('button'):
    texto = i.text
    value = '' if 'value' not in i.attrs.keys() else i.attrs['value']
    pages[texto] = {
        'link': f'{embrapa_url}?opcao={value}',
        'has_subpages': 'No', 
        'sub_pages': {},
        'download_link': {}
        }
    print(texto, value)

Apresentação opt_01
Produção opt_02
Processamento opt_03
Comercialização opt_04
Importação opt_05
Exportação opt_06
Publicação opt_07


In [4]:
# Sub Páginas

for key, values in pages.items():
    print(key)
    re = requests.get(values['link'])
    soup = BeautifulSoup(re._content)
    subpages_dict = {}
    for i in soup.find_all('button'):
        text = i.text
        btn_value = '' if 'value' not in i.attrs.keys() else i.attrs['value']
        if 'subopt' in btn_value:
            print(f"subopt: {btn_value}")
            values['has_subpages'] = 'yes' 
            subpages_dict[text] = f"{values['link']}&subopcao={btn_value}"
    values['sub_pages'] = subpages_dict


Apresentação
Produção
Processamento
subopt: subopt_01
subopt: subopt_02
subopt: subopt_03
subopt: subopt_04
Comercialização
Importação
subopt: subopt_01
subopt: subopt_02
subopt: subopt_03
subopt: subopt_04
subopt: subopt_05
Exportação
subopt: subopt_01
subopt: subopt_02
subopt: subopt_03
subopt: subopt_04
Publicação


In [12]:
def detect_delimiter(sample_line):
    """Detecta o delimitador da linha de amostra."""
    if sample_line.count(';') > sample_line.count('\t'):
        return ';'
    else:
        return '\t'

In [14]:
# Download Links
for key, values in pages.items():
    if key in ['Apresentação', 'Publicação']: continue
    if values['has_subpages'] == 'No':
        re = requests.get(values['link'])
        soup = BeautifulSoup(re._content)
        for link_element in soup.find_all('a', class_='footer_content'):
            if 'href' in link_element.attrs.keys() and 'DOWNLOAD' in link_element.text:
                values['download_link'] = [embrapa_url.replace('index.php', link_element['href'])]
                # Fazendo a requisição para baixar o conteúdo do CSV
                if 'pdf' in link_element['href']: continue
                response = requests.get(embrapa_url.replace('index.php', link_element['href']))
                # Checando se o download foi bem-sucedido
                if response.status_code == 200:
                    # Salvando o conteúdo do arquivo
                    name = link_element['href'].split('/')[-1]
                    csv_content = response.content.decode('utf-8')
                    
                    # Criando um StringIO para processar o conteúdo como um arquivo
                    csv_stream = io.StringIO(csv_content)
                    first_line = csv_stream.readline()
                    
                    # Detectando o delimitador na primeira linha
                    detected_delimiter = detect_delimiter(first_line)
                    
                    # Voltando ao início do stream para ler tudo novamente
                    csv_stream.seek(0)
                    
                    # Lendo o CSV original com o delimitador detectado
                    csv_reader = csv.reader(csv_stream, delimiter=detected_delimiter)
                    
                    # Reescrevendo o CSV com ";" como delimitador
                    with open(f"data/data_{name}", "w", newline='') as file:
                        csv_writer = csv.writer(file, delimiter=';')
                        for row in csv_reader:
                            csv_writer.writerow(row)
                    
                    print("Arquivo CSV baixado, modificado e salvo com sucesso.")
                else:
                    print(f"Erro ao baixar o arquivo: {response.status_code}")
    else:
        download_link = []
        for subpages in values['sub_pages'].values():
            re = requests.get(subpages)
            soup = BeautifulSoup(re._content)
            for link_element in soup.find_all('a', class_='footer_content'):
                if 'href' in link_element.attrs.keys() and 'DOWNLOAD' in link_element.text:
                    download_link.append(embrapa_url.replace('index.php', link_element['href']))
                    # Fazendo a requisição para baixar o conteúdo do CSV
                    if 'pdf' in link_element['href']: continue
                    response = requests.get(embrapa_url.replace('index.php', link_element['href']))
                    # Checando se o download foi bem-sucedido
                    if response.status_code == 200:
                        # Salvando o conteúdo do arquivo
                        name = link_element['href'].split('/')[-1]
                        csv_content = response.content.decode('utf-8')
                        
                        # Criando um StringIO para processar o conteúdo como um arquivo
                        csv_stream = io.StringIO(csv_content)
                        first_line = csv_stream.readline()
                        
                        # Detectando o delimitador na primeira linha
                        detected_delimiter = detect_delimiter(first_line)
                        
                        # Voltando ao início do stream para ler tudo novamente
                        csv_stream.seek(0)
                        
                        # Lendo o CSV original com o delimitador detectado
                        csv_reader = csv.reader(csv_stream, delimiter=detected_delimiter)
                        
                        # Reescrevendo o CSV com ";" como delimitador
                        with open(f"data/data_{name}", "w", newline='') as file:
                            csv_writer = csv.writer(file, delimiter=';')
                            for row in csv_reader:
                                csv_writer.writerow(row)
                        
                        print(f"Arquivo {name} CSV baixado, modificado e salvo com sucesso 1.")
                    else:
                        print(f"Erro ao baixar o arquivo: {response.status_code}")
        values['download_link'] = download_link



Arquivo CSV baixado, modificado e salvo com sucesso.
Arquivo ProcessaViniferas.csv CSV baixado, modificado e salvo com sucesso 1.
Arquivo ProcessaAmericanas.csv CSV baixado, modificado e salvo com sucesso 1.
Arquivo ProcessaMesa.csv CSV baixado, modificado e salvo com sucesso 1.
Arquivo ProcessaSemclass.csv CSV baixado, modificado e salvo com sucesso 1.
Arquivo CSV baixado, modificado e salvo com sucesso.
Arquivo ImpVinhos.csv CSV baixado, modificado e salvo com sucesso 1.
Arquivo ImpEspumantes.csv CSV baixado, modificado e salvo com sucesso 1.
Arquivo ImpFrescas.csv CSV baixado, modificado e salvo com sucesso 1.
Arquivo ImpPassas.csv CSV baixado, modificado e salvo com sucesso 1.
Arquivo ImpSuco.csv CSV baixado, modificado e salvo com sucesso 1.
Arquivo ExpVinho.csv CSV baixado, modificado e salvo com sucesso 1.
Arquivo ExpEspumantes.csv CSV baixado, modificado e salvo com sucesso 1.
Arquivo ExpUva.csv CSV baixado, modificado e salvo com sucesso 1.
Arquivo ExpSuco.csv CSV baixado, mod

In [27]:
path = 'data/'
extension = 'csv'
result = glob.glob('data/*.{}'.format(extension))
print(result)

['data/data_ProcessaSemclass.csv', 'data/data_ImpFrescas.csv', 'data/data_ProcessaViniferas.csv', 'data/data_ExpUva.csv', 'data/data_Producao.csv', 'data/data_ExpSuco.csv', 'data/data_ExpEspumantes.csv', 'data/data_ProcessaMesa.csv', 'data/data_ImpVinhos.csv', 'data/data_ImpEspumantes.csv', 'data/data_ExpVinho.csv', 'data/data_ProcessaAmericanas.csv', 'data/data_ImpPassas.csv', 'data/data_ImpSuco.csv', 'data/data_Comercio.csv']


In [34]:
import sqlite3
import pandas as pd
def conectar_sqlite():
    """ create a database connection to an SQLite database """
    conn = None
    filename = "TechChallenge01.db"
    try:
        conn = sqlite3.connect(filename)
        print(sqlite3.sqlite_version)
    except sqlite3.Error as e:
        print(e)
    finally:
        if conn:
            return conn
            

def write_table(conn, file, table_name):
    # read file
    df = pd.read_csv(file, sep=';')
    # write the data to a sqlite table
    df.to_sql(table_name, conn, if_exists='replace', index = False)

def get_files():
    path = '../data/'
    extension = 'csv'
    result = glob.glob(f'data/*.{extension}')
    print(result)
    return(result)

def save_tables_in_db():
    conn = conectar_sqlite()
    for i in get_files():
        write_table(conn=conn, file=i, table_name=i.replace("data/data_", "").replace(".csv", ""))

In [38]:
get_tables_from_db()

3.31.1
['data/data_ProcessaSemclass.csv', 'data/data_ImpFrescas.csv', 'data/data_ProcessaViniferas.csv', 'data/data_ExpUva.csv', 'data/data_Producao.csv', 'data/data_ExpSuco.csv', 'data/data_ExpEspumantes.csv', 'data/data_ProcessaMesa.csv', 'data/data_ImpVinhos.csv', 'data/data_ImpEspumantes.csv', 'data/data_ExpVinho.csv', 'data/data_ProcessaAmericanas.csv', 'data/data_ImpPassas.csv', 'data/data_ImpSuco.csv', 'data/data_Comercio.csv']
data/data_ProcessaSemclass.csv
data/data_ImpFrescas.csv
data/data_ProcessaViniferas.csv
data/data_ExpUva.csv
data/data_Producao.csv
data/data_ExpSuco.csv
data/data_ExpEspumantes.csv
data/data_ProcessaMesa.csv
data/data_ImpVinhos.csv
data/data_ImpEspumantes.csv
data/data_ExpVinho.csv
data/data_ProcessaAmericanas.csv
data/data_ImpPassas.csv
data/data_ImpSuco.csv
data/data_Comercio.csv


In [45]:
df = pd.read_csv("data/data_ProcessaAmericanas.csv", sep=';')
df

,id,control,cultivar,1970,1971,1972,1973,1974,1975,1976,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,1,TINTAS,TINTAS,284285642,193695427,187251954,161593877,295404693,282003228,222289729,...,471555903,537592720,235356261,600605876,519506589,nd,376136313,394224543,*,502666358
1,2,ti_Bacarina,Bacarina,82899,106962,67464,58690,138158,101454,57297,...,3900,0,0,0,0,nd,0,0,*,0
2,3,ti_Bailey,Bailey,0,0,0,0,0,0,0,...,991449,963159,442784,1370092,539742,nd,534981,4092669,*,587066
3,4,ti_Bordo,Bordo,7242197,7227090,6530686,5584243,13341412,16023998,12725233,...,113008320,137467196,60976531,160146475,158405972,nd,129978861,117655879,*,154310837
4,5,ti_Bourdin (S),Bourdin (S),0,0,0,0,0,0,0,...,0,0,0,0,0,nd,0,0,*,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,58,br_SeyveVillard,Seyve Villard,0,0,145444,0,0,11830,0,...,700473,657086,312824,586955,462683,nd,313488,0,*,0
58,59,br_SeyveVillard5276,Seyve Villard 5276,0,0,0,0,0,0,0,...,107521,91960,60000,206295,139575,nd,57423,97368,*,105771
59,60,br_SeyveVillard12375,Seyve Villard 12375,0,0,0,0,0,0,0,...,0,0,0,0,0,nd,0,61942,*,329582
60,61,br_TardiadeCaxias,Tardia de Caxias,0,0,0,0,0,0,0,...,14890,17711,3440,6170,5380,nd,8150,33870,*,0


In [44]:
df = pd.read_csv("data/data_Comercio.csv", sep=';')
df


,id,control,Produto,1970,1971,1972,1973,1974,1975,1976,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,1,VINHO DE MESA,VINHO DE MESA,98327606,114399031,118377367,116617910,94173324,108031792,139238614,...,206404427,209198468,166769622,176059959,177186273,180446489,215557931,210012238,187939996,187016848
1,2,vm_Tinto,Tinto,83300735,98522869,101167932,98196747,77167303,91528090,116407222,...,178250072,182028785,146646365,154309442,155115499,158519218,189573423,185653678,165067340,165097539
2,3,vm_Rosado,Rosado,107681,542274,7770851,8425617,8891367,7261777,11748047,...,1419855,1409002,1391942,1097426,1972944,1265435,1394901,1931606,2213723,2520748
3,4,vm_Branco,Branco,14919190,15333888,9438584,9995546,8114654,9241925,11083345,...,26734500,25760681,18731315,20653091,20097830,20661836,24589607,22426954,20658933,19398561
4,5,VINHO FINO DE MESA,VINHO FINO DE MESA,4430629,4840369,5602091,7202830,7571802,8848303,14095648,...,20424983,20141631,19630158,15874354,14826143,15684588,24310834,27080445,21533487,18589310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,58,ou_Vinho_base,Vinho base para espumantes,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58,59,ou_Vinho_composto,Vinho composto,3386173,3645774,3921573,4168841,3670736,3493104,5429800,...,450121,413735,377685,485944,249165,366549,63625,227116,32000,981
59,60,ou_Vinho_licoroso,Vinho licoroso,3013056,2475632,2407816,2443944,1921867,2205036,2474364,...,616904,510975,546189,496746,491165,465964,362340,376597,385006,421974
60,61,ou_Vinho_leve,Vinho leve,0,0,0,0,0,0,0,...,0,0,0,0,198,306,396,0,27,132064


In [42]:
conn = conectar_sqlite()
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(query, conn)
tables

3.31.1


,name
0,ProcessaSemclass
1,ImpFrescas
2,ProcessaViniferas
3,ExpUva
4,Producao
5,ExpSuco
6,ExpEspumantes
7,ProcessaMesa
8,ImpVinhos
9,ImpEspumantes
